## Deliverable 2. Create a Customer Travel Destinations Map.

## import dependencies

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

## import weather database csv from deliverable 1

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\limag\Desktop\Module6\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Date
0,0,Albany,US,42.6001,-73.9662,37.67,76,100,1.01,overcast clouds,2021-12-02 04:39:44
1,1,Qaanaaq,GL,77.4840,-69.3632,4.39,68,30,7.74,scattered clouds,2021-12-02 04:39:45
2,2,Rikitea,PF,-23.1203,-134.9692,75.79,77,5,19.37,light rain,2021-12-02 04:39:47
3,3,La Paz,BO,-16.5000,-68.1500,44.58,93,40,4.61,fog,2021-12-02 04:39:47
4,4,East London,ZA,-33.0153,27.9116,65.34,100,90,6.91,mist,2021-12-02 04:39:47


## write 2 input statements

variable = float(input(string question)) 
the question uses the input method to prompt user, the float() method converts the input given by user as a float datatype

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [5]:
min_temp

60.0

In [6]:
max_temp

80.0

## loc() method to filter the city_data_df dataframe 

In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description,Date
2,2,Rikitea,PF,-23.1203,-134.9692,75.79,77,5,19.37,light rain,2021-12-02 04:39:47
4,4,East London,ZA,-33.0153,27.9116,65.34,100,90,6.91,mist,2021-12-02 04:39:47
5,5,Raymondville,US,26.4815,-97.7831,73.24,85,100,5.99,overcast clouds,2021-12-02 04:39:48
8,8,Kathmandu,NP,27.7167,85.3167,62.82,63,75,3.44,broken clouds,2021-12-02 04:39:41
9,9,Xapuri,BR,-10.6517,-68.5044,74.88,87,99,3.24,overcast clouds,2021-12-02 04:39:49
15,15,Coahuayana,MX,18.7333,-103.6833,73.54,89,0,3.71,clear sky,2021-12-02 04:39:51
16,16,Chabahar,IR,25.2919,60.6430,76.59,38,0,2.21,clear sky,2021-12-02 04:39:51
22,22,Codrington,AU,-38.2667,141.9667,62.08,73,77,22.01,broken clouds,2021-12-02 04:39:53
24,24,Tessalit,ML,20.1986,1.0114,68.67,18,30,8.97,scattered clouds,2021-12-02 04:39:53
26,26,Mataura,NZ,-46.1927,168.8643,64.47,49,100,4.90,overcast clouds,2021-12-02 04:37:04


In [8]:
# check how many total cities fit the bill
len(preferred_cities_df)

243

## check for nulls or empty rows

In [9]:
preferred_cities_df.count()

Unnamed: 0             243
City                   243
Country                242
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Weather Description    243
Date                   243
dtype: int64

In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
Date                   0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
    # no empty rows

## new dataframe to hold all hotel names

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Rikitea,PF,75.79,light rain,-23.1203,-134.9692,
4,East London,ZA,65.34,mist,-33.0153,27.9116,
5,Raymondville,US,73.24,overcast clouds,26.4815,-97.7831,
8,Kathmandu,NP,62.82,broken clouds,27.7167,85.3167,
9,Xapuri,BR,74.88,overcast clouds,-10.6517,-68.5044,
15,Coahuayana,MX,73.54,clear sky,18.7333,-103.6833,
16,Chabahar,IR,76.59,clear sky,25.2919,60.6430,
22,Codrington,AU,62.08,broken clouds,-38.2667,141.9667,
24,Tessalit,ML,68.67,scattered clouds,20.1986,1.0114,
26,Mataura,NZ,64.47,overcast clouds,-46.1927,168.8643,


# Requesting data from google using api

In [13]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat}, {lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    google_places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(google_places_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("hotel not found skipping")            

hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping
hotel not found skipping


## removing empty strings

In [40]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
9,Xapuri,BR,74.88,overcast clouds,-10.6517,-68.5044,Pousada dos Chapurys
45,Aguimes,ES,63.23,broken clouds,27.9054,-15.4461,Hotel Villa de Aguimes
58,Luderitz,NaN,60.48,clear sky,-26.6481,15.1594,LÜDERITZ NEST HOTEL
62,Te Anau,NZ,72.41,overcast clouds,-45.4167,167.7167,Kingsgate Hotel Te Anau
88,Aracruz,BR,74.41,scattered clouds,-19.8203,-40.2733,OYO Hotel Iramar
137,Airai,TL,76.98,broken clouds,-8.9266,125.4092,Juvinal Martin
180,Arraial Do Cabo,BR,72.86,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
188,Nelson Bay,AU,79.77,few clouds,-32.7167,152.1500,Mantra Nelson Bay
231,Ribeira Grande,PT,64.33,clear sky,38.5167,-28.7000,Quinta da Meia Eira
250,Lingao,CN,70.79,scattered clouds,19.9103,109.6859,Lingao Ditai Inn


# creating csv

In [42]:
# 8a. Create the output File (CSV)
output_file = "weatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index_label="City_ID")

# creating heat map with google maps

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))